In [ ]:
import ants
import nibabel as nib
import numpy as np
from nilearn import datasets, image
import glob
import os

data_root = '/workspaces/data/brain_meningioma/bet'
testImages = glob.glob(os.path.join(data_root,'b_test/*bet.nii.gz'))
# testGTVs = 
testImage = testImages[2]

# === Step 1: Load your MRI and mask ===
moving_img = ants.image_read(testImage)  # MRI (with or without skull)
moving_mask = ants.image_read(testImage.replace('_bet','_gtv_mask'))  # binary mask (e.g., tumor)
print(testImage, moving_mask.sum())

# === Step 2: Download and load MNI152 template ===
mni = ants.image_read(ants.get_ants_data("mni"))
print("Loaded MNI shape:", mni.shape)
print("MNI spacing:", mni.spacing)

# === Step 3: Register your image to MNI ===
reg = ants.registration(fixed=mni, moving=moving_img, type_of_transform="SyN")
registered_img = reg["warpedmovout"]

# === Step 4: Transform your mask into MNI space ===
registered_mask = ants.apply_transforms(
    fixed=mni,
    moving=moving_mask,
    transformlist=reg["fwdtransforms"],
    interpolator="nearestNeighbor"
)

# Save registered mask if needed
# ants.image_write(registered_mask, "mask_in_mni.nii.gz")

# === Step 5: Load Harvard-Oxford atlas ===
atlas = datasets.fetch_atlas_harvard_oxford("cort-maxprob-thr25-1mm")
atlas_img = nib.load(atlas.filename)
atlas_data = atlas_img.get_fdata()
atlas_labels = atlas.labels

# === Step 6: Resample mask to match atlas ===
# registered_mask_nib = nib.Nifti1Image(registered_mask.numpy(), affine=registered_mask.affine)
# registered_mask_nib = ants.to_nibabel(registered_mask)
mask_data = registered_mask.numpy()
spacing = registered_mask.spacing
direction = registered_mask.direction
origin = registered_mask.origin
affine = np.eye(4)
for i in range(3):
    for j in range(3):
        affine[i, j] = direction[i, j] * spacing[j]
    affine[i, 3] = origin[i]
registered_mask_nib = nib.Nifti1Image(mask_data, affine=affine)


resampled_mask = image.resample_to_img(registered_mask_nib, atlas_img, interpolation='nearest')
mask_data = resampled_mask.get_fdata()

# === Step 7: Extract overlapped regions ===
label_ids = atlas_data[mask_data > 0]
unique_ids = np.unique(label_ids.astype(int))
region_ids = unique_ids[unique_ids > 0]
region_names = [atlas_labels[i] for i in region_ids]

# === Step 8: Print regions ===
print("=== Detected Brain Regions Overlapping with Mask ===")
for name in region_names:
    print(f"- {name}")


/workspaces/data/brain_meningioma/bet/b_test/BraTS-MEN-RT-0022-1_t1c_bet.nii.gz 1291.0
Loaded MNI shape: (182, 218, 182)
MNI spacing: (1.0, 1.0, 1.0)


[fetch_atlas_harvard_oxford] Dataset found in /home/vscode/nilearn_data/fsl

=== Detected Brain Regions Overlapping with Mask ===
- Precuneous Cortex


/tmp/ipykernel_13686/3685079099.py:59: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_mask = image.resample_to_img(registered_mask_nib, atlas_img, interpolation='nearest')
/tmp/ipykernel_13686/3685079099.py:59: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  resampled_mask = image.resample_to_img(registered_mask_nib, atlas_img, interpolation='nearest')


In [39]:
mask_data.sum()

512.0

In [ ]:
import ants
import nibabel as nib
import numpy as np
from nilearn import datasets, image
import glob
import os

# === Step 1: Load your MRI and mask ===
data_root = '/workspaces/data/brain_meningioma/bet'
testImages = sorted(glob.glob(os.path.join(data_root, 'b_test/*bet.nii.gz')))
if not testImages:
    raise FileNotFoundError(f"No files found in {os.path.join(data_root, 'b_test/*bet.nii.gz')}")
testImage = testImages[3]
print("Selected image:", testImage)

mask_file = testImage.replace('_bet', '_gtv_mask')
if not os.path.exists(mask_file):
    raise FileNotFoundError(f"Mask file not found: {mask_file}")

moving_img = ants.image_read(testImage)
moving_mask = ants.image_read(mask_file)
print("Mask values:", np.unique(moving_mask.numpy()))
print(testImage, moving_mask.sum())

# === Step 2: Download and load MNI152 template ===
mni_dataset = datasets.fetch_icbm152_2009()
mni = ants.image_read(mni_dataset.t1)
print("MNI spacing:", mni.spacing, "shape:", mni.shape)

# === Step 3: Register your image to MNI ===
reg = ants.registration(fixed=mni, moving=moving_img, type_of_transform="SyNRA")
registered_img = reg["warpedmovout"]

# === Step 4: Transform your mask into MNI space ===
moving_mask = ants.threshold_image(moving_mask, low_thresh=0.5, high_thresh=1e9)
registered_mask = ants.apply_transforms(
    fixed=mni,
    moving=moving_mask,
    transformlist=reg["fwdtransforms"],
    interpolator="nearestNeighbor"
)

# === Step 5: Load Harvard-Oxford atlas ===
atlas = datasets.fetch_atlas_harvard_oxford("cort-maxprob-thr25-1mm")
atlas_img = nib.load(atlas.filename)
atlas_data = atlas_img.get_fdata()
atlas_labels = atlas.labels

# === Step 6: Resample mask to match atlas ===
try:
    registered_mask_nib = registered_mask.to_nibabel()
except AttributeError:
    # Fallback if to_nibabel() is unavailable
    mask_data = registered_mask.numpy()
    spacing = registered_mask.spacing
    direction = registered_mask.direction
    origin = registered_mask.origin
    affine = np.eye(4)
    for i in range(3):
        for j in range(3):
            affine[i, j] = direction[i, j] * spacing[j]
        affine[i, 3] = origin[i]
    registered_mask_nib = nib.Nifti1Image(mask_data, affine=affine)

resampled_mask = image.resample_to_img(registered_mask_nib, atlas_img, interpolation='nearest')
mask_data = resampled_mask.get_fdata()
if not np.all(np.isin(np.unique(mask_data), [0, 1])):
    print("Warning: Resampled mask is not binary, binarizing...")
    mask_data = (mask_data > 0.5).astype(np.float64)

# === Step 7: Extract overlapped regions ===
label_ids = atlas_data[mask_data > 0]
unique_ids = np.unique(label_ids.astype(int))
region_ids = unique_ids[unique_ids > 0]
if np.max(unique_ids) >= len(atlas_labels):
    raise ValueError(f"Atlas label index {np.max(unique_ids)} exceeds number of labels ({len(atlas_labels)})")
region_names = [atlas_labels[i] for i in region_ids]

# === Step 8: Print regions ===
print("=== Detected Brain Regions Overlapping with Mask ===")
if not region_names:
    print("No overlapping regions found. Check registration and resampling.")
else:
    for name in region_names:
        print(f"- {name}")


Selected image: /workspaces/data/brain_meningioma/bet/b_test/BraTS-MEN-RT-0022-1_t1c_bet.nii.gz
Mask values: [0. 1.]
/workspaces/data/brain_meningioma/bet/b_test/BraTS-MEN-RT-0022-1_t1c_bet.nii.gz 1291.0


[fetch_icbm152_2009] Dataset found in /home/vscode/nilearn_data/icbm152_2009

MNI spacing: (1.0, 1.0, 1.0) shape: (197, 233, 189)


[fetch_atlas_harvard_oxford] Dataset found in /home/vscode/nilearn_data/fsl

/tmp/ipykernel_1684/3559323188.py:65: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_mask = image.resample_to_img(registered_mask_nib, atlas_img, interpolation='nearest')
/tmp/ipykernel_1684/3559323188.py:65: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  resampled_mask = image.resample_to_img(registered_mask_nib, atlas_img, interpolation='nearest')


=== Detected Brain Regions Overlapping with Mask ===
- Lateral Occipital Cortex, superior division
- Lateral Occipital Cortex, inferior division


In [43]:
mask_data.sum()

523.0

In [20]:
registered_mask

ANTsImage (RPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (182, 218, 182)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-90.0, 126.0, -72.0)
	 Direction  : [ 1.  0.  0.  0. -1.  0.  0.  0.  1.]